In [ ]:
import cv2
import numpy as np

# capture the video
cap = cv2.VideoCapture(0) # webcam

# read the frame
ret, frame = cap.read()

# convert to grayscale for better tracking
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# haarcascade for face
face_casc = cv2.CascadeClassifier('HaarCascades/haarcascade_frontalface_default.xml')
face_rects = face_casc.detectMultiScale(gray, 1.3, 5)

# convert list to touple
face_x, face_y, w, h = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)

# set the region of interest for tracking
roi = frame[face_y:face_y+h,
            face_x:face_x+w]

# hsv color maping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# histogram to target on each frame
roi_hist = cv2.calcHist([hsv_roi],
                       [0],
                       None,
                       [180],
                       [0, 180])

# normalize the histogram,cuz when we apply camshift wehave to calculate meanshift first
cv2.normalize(roi_hist,
             roi_hist,
             0,
             255,
             cv2.NORM_MINMAX)

# termination criteria
term_crit = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS,10,1)


# while loop
while True:
    # read the frame
    ret, frame = cap.read()
    
    # if statement
    if ret == True:
        
        #frame in hsv
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # calculate thje base of ROI
        dest = cv2.calcBackProject([hsv],
                                  [0],
                                  roi_hist,
                                  [0, 180],
                                  1)
        
        # camshift to get the new coordinates
        ret, track_window = cv2.CamShift(dest,
                                        track_window,
                                        term_crit)
        
        # draw a rectangle
        pts = cv2.boxPoints(ret) # points in the image
        pts = np.int0(pts)
        img = cv2.polylines(frame,
                           [pts],
                            True,
                           (0,255,255),
                           3)
        
        # open new window and display
        cv2.imshow('Cam Shift', img)
        
        #close window
        if cv2.waitKey(15) & 0xFF==ord('q'):
            break
    
    # else statement
    else:
        break
cap.release()
cv2.destroyAllWindows()

